In [ ]:
#Specify code location
import sys
sys.path.append('/home/general/TEMcode/tomography_gpu/')

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import os
import fnmatch
import contexttimer
import TEM_misc, TEM_recon
import sys
import h5py
#For Lahvahn
sys.path.append('D:\\David\\libwallerlab\\python\\libwallerlab\\tomography_gpu')
#For YONGHUAN_PC
sys.path.append('L:\\David\\libwallerlab\\python\\libwallerlab\\tomography_gpu')
#For COSMOS
sys.path.append('/home/david/lahvahn_david/libwallerlab/python/libwallerlab/tomography_gpu/')
#For others
sys.path.append('T:\\David\\libwallerlab\\python\\libwallerlab\\tomography_gpu')
from opticaltomography.opticsutil import compare3DStack, show3DStack
from opticaltomography.opticsalg import PhaseObject3D, TomographySolver, AlgorithmConfigs

### Specify Parameters

#### Secify the experiment numbers

In [ ]:
exp_i = -1
flag_add_noise = True

#### Datapath

In [ ]:
# Specify datapath
rec_dir     = "/home/general/TEMrec/"
mea_dir     = "/home/general/TEMdata/"

#### All dataset

In [ ]:
# All datasets available (Preprocessed)
datasets = {
    "sim_sio2_full"                : "TEM_simulation_480_SiO2_py.mat",
    "sim_sio2_more_measure"        : "TEM_simulation_480_SiO2_full_py.mat",
    "sim_sio2_full_vacant"         : "TEM_simulation_480_SiO2_vacancy_py.mat",    
}

In [ ]:
# Reconstruction
for file in os.listdir(rec_dir):
    if fnmatch.fnmatch(file, "exp%s*" % (str(exp_i))):
        print(file)
        reconstruction = sio.loadmat(rec_dir+file, squeeze_me=True)
        break
# Measurements
dset = datasets[str(reconstruction["opts"]["dset"])]
print(dset)
measurements = sio.loadmat(mea_dir+dset)

#### Forward model

In [ ]:
if "intensity_ms" not in measurements:
    if "field_measure" not in measurements:
        intensity_measure = measurements["amplitude_measure"] ** 2
    else:
        intensity_measure = np.abs(measurements["field_measure"]) ** 2
measurements["intensity_measure"] = intensity_measure
TEM_obj = TEM_recon.TEM_recon_gpu(measurements, reconstruction["opts"])

In [ ]:
with contexttimer.Timer() as timer:
    TEM_obj.solver_obj._x = reconstruction["obj_final"]
    amplitude_measure = np.squeeze(TEM_obj.solver_obj.forwardPredict())
    print("Time used:", timer.elapsed)

#### Add noise

In [ ]:
if flag_add_noise:
    dose = int(reconstruction["opts"]["dose_per_pixel"])
    temp = measurements["intensity_measure"]
    temp = np.random.poisson(temp * dose).astype(float)
    temp /= dose
    min_val = np.min(temp[:])
    max_val = np.max(temp[:])    
    print("Showing noisy intensity measurements.")
    print("min:", min_val)
    print("max:", max_val)
show3DStack(np.squeeze(temp[:,:,2,:]), clim=(min_val,max_val))

#### Compare 3D Stack

In [ ]:
true_meas = np.abs(np.squeeze(measurements["intensity_measure"][:,:,0,:]**0.5))
min_val = np.min(true_meas[:])
max_val = np.max(true_meas[:])
print("min:", min_val)
print("max:", max_val)
diff_obj = np.abs(true_meas - np.abs(np.squeeze(amplitude_measure[:,:,0,:])))
compare3DStack(true_meas, np.abs(np.squeeze(amplitude_measure[:,:,0,:])), clim=(min_val,max_val))

In [ ]:
min_val = np.min(diff_obj[:])
max_val = np.max(diff_obj[:])
print("min:", min_val)
print("max:", max_val)
show3DStack(diff_obj, clim=(min_val,max_val))

In [ ]:
true_meas = np.squeeze(measurements["intensity_measure"][:,:,2,:]**0.5)
min_val = np.min(true_meas[:])
max_val = np.max(true_meas[:])
print("min:", min_val)
print("max:", max_val)
diff_obj = np.abs(true_meas - np.abs(np.squeeze(amplitude_measure[:,:,2,:])))
compare3DStack(true_meas, np.squeeze(amplitude_measure[:,:,2,:]), clim=(min_val,max_val))

In [ ]:
min_val = np.min(diff_obj[:])
max_val = np.max(diff_obj[:])
print("min:", min_val)
print("max:", max_val)
show3DStack(diff_obj, clim=(min_val,max_val))